In [1]:
import json
import pandas as pd
from tqdm import tqdm

raw = []
with open('data/nsmc/raw.json', 'r') as fp:
    buf = ''
    for line in tqdm(fp.readlines()):
        line = line.strip()            
        
        if line == '][':
            buf += ']'
            raw += json.loads(buf)
            
            buf = '['
            continue
            
        buf += line.strip()

100%|██████████| 5713963/5713963 [00:02<00:00, 1913893.50it/s]


In [7]:
raw_df = pd.DataFrame(raw)
raw_df.head()

,review_id,movie_id,author,review,rating,date
0,10274036,100015,wldu****,Good 완전 재밌어요,10,15.08.25
1,10227849,100015,nkjh****,재미 만점..아이와함께,9,15.08.14
2,10226738,100015,alex****,"픽사처럼 감동이 있는것도 아니고,,, 솔직히 셔먼이 ""나도 개다"" 하고 다른 사람들...",5,15.08.14
3,10200946,100015,mj46****,정말 잼있어용 ㅎㅎ 수고하세요,10,15.08.08
4,10169818,100015,h225****,보면서도 상상을 하게하는 영화였습니다^^,10,15.08.01


In [ ]:
raw_df.to_json(
    'data/nsmc.json.bz2',
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)

In [ ]:
raw_df[raw_df['review_id'] == '9976970']

In [2]:
import pandas as pd

filename = {
    'test': 'data/nsmc/ratings_test.txt',
    'train': 'data/nsmc/ratings_train.txt',
} 

data = {
    'test': pd.read_csv(filename['test'], sep='\t'),
    'train': pd.read_csv(filename['train'], sep='\t'),
}

In [10]:
idx = {}

for role in data:
    df = data[role]
    for i, row, in tqdm(df.iterrows(), total=len(df)):
        idx[str(row['id'])] = {
            'label': row['label'],
            'role': role
        }

100%|██████████| 150000/150000 [00:12<00:00, 12360.24it/s]


In [12]:
for item in tqdm(raw):
    review_id = item['review_id']
    if review_id in idx:
        item.update(idx[review_id])

100%|██████████| 712371/712371 [00:00<00:00, 2131407.30it/s]


In [15]:
raw_df = pd.DataFrame(raw)

raw_df.fillna('', inplace=True)
raw_df.head()

,review_id,movie_id,author,review,rating,date,label,role
0,10274036,100015,wldu****,Good 완전 재밌어요,10,15.08.25,,
1,10227849,100015,nkjh****,재미 만점..아이와함께,9,15.08.14,1,test
2,10226738,100015,alex****,"픽사처럼 감동이 있는것도 아니고,,, 솔직히 셔먼이 ""나도 개다"" 하고 다른 사람들...",5,15.08.14,,
3,10200946,100015,mj46****,정말 잼있어용 ㅎㅎ 수고하세요,10,15.08.08,,
4,10169818,100015,h225****,보면서도 상상을 하게하는 영화였습니다^^,10,15.08.01,1,train


In [22]:
raw_df.to_json(
    'data/nsmc-all.json.bz2',
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)

In [21]:
raw_df[raw_df['role'] != ''].to_excel('data/nsmc-datasets.xlsx', index=False)